## System Design
In this section, we will talk about various system design to build robust software.

### 1. Circuit Braker
A circuit breaker is a design pattern used to detect failures and stop making requests to a failing service, giving it time to recover. It’s like an automatic switch: if something goes wrong too often, it stops the flow to prevent overload. It brings in resiliency in the application allowing it to gracefully exit and allow the system to recover.

🔌 Think of it like an electric circuit:
    - Closed: Everything is working, requests flow as normal.

    - Open: Service is failing, requests are blocked immediately.

    - Half-Open: Service is being tested. If successful, it returns to Closed.

#### Why do I need it?

In microservices or distributed systems:

- If one external service is down or slow, you don’t want to keep hitting it and slow down or crash your whole app.

- Helps prevent cascading failures, keeps your system resilient and responsive.

- Useful when calling APIs, databases, payment gateways, etc.

<img src="./_images/Circuit-Breaker-Design-Pattern.png" alt="Circuit Braker" width="1000" height="500">


#### Use Case:

Let’s say you have a Node.js REST API calling an external weather service. If that service is down, you don't want to keep calling it—use a circuit breaker!

#### Implementation Options

    - Service Mesh Layer: Tools like Istio, Linkerd, or Envoy provide circuit breaker features out of the box. Centralized control over all services. No code change is neeed.

    - API Gateway / Reverse Proxy: Gateways like Azure API Management, Kong, NGINX, or AWS API Gateway can enforce circuit breaker-like behavior. Configure API Management to limit calls to a backend if it starts failing or slows down. 

    - K8 Replica Set is also a type of circuit braker. It removes the unhealthy containers.

    - Programming: You implement circuit braker through coding. Many 3rd packages are also available. 
        - Polly Nuget Package: It supports Circuit Braker, Retry, Timeout, Bulkhead isolation, and Fallback.
        - Resiliency4j Maven Package: It implements CircuitBreaker, Retry, RateLimiter, Bulkhead, etc.
        - NPM opossum package: Circuit breaker library for Node.js.



### 2. Ambassador
An **Ambassador** is a helper component (usually deployed alongside your service) that acts as a proxy between your application and external systems. It manages outbound requests on behalf of the main service.

It’s called "ambassador" because it "represents" your service to the outside world, handling duties that aren’t core to the business logic, like logging, authentication, encryption, and controlling traffic (ingress / egress).

#### Why do I need it?

**_Separation of concerns_**: Keeps your main service code clean by offloading cross-cutting responsibilities.

**_Observability & reliability_**: Enables better logging, monitoring, retries, and circuit breakers.

**_Security_**: Handles authentication and encryption consistently.

**_Consistency_**: Applies standard behavior across services (e.g., all outgoing calls are logged the same way).

**_Flexibility_**: Makes it easy to update the proxy logic without touching app code.

<img src="./_images/ambassador.png" alt="Circuit Braker" width="1000" height="400">

**You can always introduce this pattern to an already existing app or legacy app without making changes to the app code.**

#### Use Case:

You have a Node.js service that needs to call an external weather API (like OpenWeatherMap). You want to add:

    - API key injection
    - Retries
    - Timeouts
    - Request logging

Instead of putting all this in your main service code, you use a local proxy (ambassador) that handles those concerns. The Node.js app only makes simple HTTP calls to the proxy.


#### Implementation Options

    

### 3. Retry
An **Retry** pattern automatically re-attempts a failed operation, assuming the failure is temporary (e.g., network glitch, short service hiccup like throttling).

#### Why do I need it?

**_Resilence_**: To improve resilience and user experience.

**__**: To avoid failing fast on temporary issues that would resolve on their own (e.g., brief downtime or latency spikes).


#### Use Case:

You're calling a 3rd-party payment API (https://api.payment.com/charge) to charge a customer.
Sometimes the API fails with a timeout or 500 error due to high load.
You don't want to immediately give up — you'd like to retry 3 times before returning an error to the user.


#### Implementation Options
- API Gateway
- Service Mesh
- Application Code
    

### 4. PubSub
The Pub/Sub (Publish/Subscribe) pattern is a messaging pattern used widely in modern distributed systems. There are 4 components in the Pub/Sub pattern:

**_1. Publisher_**: Is the server / service which is triggering an event by posting a message into a TOPIC. 

**_2. Subscriber_**: Is the client listening to the events from the TOPICs. 

**_3. Message Broker_**: Is the client listening to the events from the TOPICs. 

**_3. Topic_**: TOPIC is a channel where you post a specific type of message. E.g. One TOPIC for Order_Event, and another TOPIC for Fullfillment_Event.

**_4. Message_**: Is the actual message. 

<img src="./_images/pubsub.png" alt="Pub Sub" width="1000" height="400">


#### Why do I need it?

**_Loose Copuling_**: To improve resilience and user experience.

**_Scalability_**: To improve resilience and user experience.

**_Asynchronous Flow_**: To improve resilience and user experience.

**_Reusability_**: To improve resilience and user experience.

**_Isolation_**: To improve resilience and user experience.


#### Use Case:

You're calling a 3rd-party payment API (https://api.payment.com/charge) to charge a customer.
Sometimes the API fails with a timeout or 500 error due to high load.
You don't want to immediately give up — you'd like to retry 3 times before returning an error to the user.

#### Messaging Model

**_Queue (Point to Point Messaging)_**: This is point to point communication between one publisher and one subscriber. It's an one-to-one decoupled communication. 

**_Publish / Subscribe_**: This is used in the scenario where you have one publisher, and multiple subscribers receiving independent copies of the message. Each subscriber processes the message independently. 

**_Event Streaming_**: Event Streaming is model where events are continosuly procduced and written to stream. Each event is persisted and ordered in the stream. Multiple consumers can independently read from different points in the stream. 
Examples:
    - Telemetry & Logging: IoT sensors send temperature, humidity, and location --> steam to Azure Event Hubs --> processed in real-time by Azure Stream Analytics or Databricks.
    - Audit Trail & Replay: Every financial transactions is logged. If a failure occurs, the system can replay the stream from a known timestamp to reconcile data.
    - Real-Time Analytics: User clicks/events from a website is streamed and analyzed in near real-time to generate dashboards and user behaviour insights.
    - Data Lake Ingestion: All application events and logs are streamed and dumped into data lake for future analysis and machine learning. 

**_Push vs Pull_**: In Push model, the broker delivers message to consumers automatically. E.g. (Event Gid working in conjuction with Webhooks or Azure Functions). 
In Pull model, the consumers poll/pull messages on demand. E.g. Service Bus, Event Hubs, Storage Queue

**_Delivery Guarantees_**: 
- At Most Once: Message is delivered once or not at all (no retry). E.g. Event Grid
- At Least Once: Message is retried until acknowledged (can be duplicated). E.g. Service Bus, Event Hubs, Storage Queue
- Exactly Once: Message is delivered exactly once (very rare, complex). Service Bus with sessions and deduplication


#### Implementation Options
- Redis
- Kafka
- Azure Services

#### Azure Services for Event Driven Architecture
**_Azure Event Grid_**: 
    - It is used for pub-sub event routing. 
    - It uses PUSH delivery to deliver message to Webhook or function app.
    - It supports multiple consumers.
    - Retains message for 24 hours.
    - Does basic re-try for Webhook.
    - High Throughput
    - No duplicate handling
    
**_Azure Event Hub_**: It is high speed event streaming solution suitable for streaming requirements.
    - It is an event streaming service. Ideal for telemetry, logs, and real-time ingestion.
    - It supports PULL delivery.
    - It supports multiple consumers.
    - Support replay. (Failed transactions can be replayed from a timestamp)
    - Reatins message upto 90 days.
    - No retry support.
    - No duplicate handling
    - Supports HTTP, AQMS and Kafka protocol
    - Very high throughput

**_Azure Service Bus_**: It is a **Message Broker**. It's reliable enterprise messaging (Queue / Topics). Queue is used for single subscriber, and Topic for multiple subscriber sutuation.
    - It provides both Message Queue and Pub-Sub (TOPIC) support.
    - Ideal for defining business workflow.
    - No replay support.
    - Provides RETRY support.
    - Retains message for 7 days.
    - Handles duplicate well.
    - It uses PULL delivery

**_Azure Storage Queue_**: It is low-cost message queuing solution. 
    - It is Basic Queue.
    - Ideal for backend job async processing.
    - It uses PULL delivery (No Push)
    - Retains message for 7 days.

**_Azure IoT Hub_**: It is also a event streaming solution. Specifically designed for IoT devices.

**_Azure SignalR Service_**: It is real-time messaging solution that uses websocket and keeps a connection always opened from the frontend to the server. Suitable for stock monitoring. 

**_Kafka_**:
    - Kafka is an open source event streaming solution from Apache and comparable with Azure Event Hub.
    - Azure Event Hub is a azure managed service. It's better for cloud-native solutions running on Azure.
    - Azure Event Hub provides security using Azure AD, and observability using azure monitors.
    - Disaster recovery is easier to setup and use.
    - On ther handa, Kafka needs a lot of maintenace support since it's not a managed service. 
    - Unlike Event Hub, Kafka stores the message for indefinite period.
    - Kafka also servers as Message broker simillar to Service Bus.
        
**_Redis_**:
    - Redis is primarily a caching solution. It also provides message-broker feature.

### 5. Repository Pattern
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

#### Why do I need it?

**_Resilence_**: To improve resilience and user experience.

**__**: To avoid failing fast on temporary issues that would resolve on their own (e.g., brief downtime or latency spikes).


#### Use Case:

xxxxxxxxxxxxxxxxxxxxxxxxxxxx


#### Implementation Options

### 6. Dependency Injection
**Dependency injection (DI)** is a software design pattern where an object receives its dependencies from an external source rather than creating them itself.

It involves 4 components:
**_Client_**: is the component or class that depends on the services provided by another class or module.

**_Service_**: is the component or class that provides a particular functionality or service that the client depends on.

**_Injector_**: is the one that instanciates the servive class, and injects into the Client.

**_Interface_**: defines the contract or set of methods that a service must implement.

**Sample Code without DI**
```javascript
```

**Sample Code with DI (Manual Injection)**
It is called Manual injection, because we are instantiating the object and sending to client. In morder frameworks like .netcore, java, nestjs, the injection happens automatically.
```javascript
// Interface: Interface for different notification providers
class NotificationProvider {
    sendNotification(message, recipient) {
        throw new Error('sendNotification method must be implemented');
    }
}

// Service1: Concrete implementations
class EmailProvider extends NotificationProvider {
    sendNotification(message, recipient) {
        // Send email logic
        console.log(`Sending email to ${recipient}: ${message}`);
    }
}

// Service2: Concrete implementations
class SMSProvider extends NotificationProvider {
    sendNotification(message, recipient) {
        // Send SMS logic
        console.log(`Sending SMS to ${recipient}: ${message}`);
    }
}

// Client: Refactored NotificationService with dependency injection
class NotificationService {
    constructor(NotificationProvider notificationProvider) {
        this.notificationProvider = notificationProvider; // Inject dependency
    }

    sendNotification(message, recipient) {
        this.notificationProvider.sendNotification(message, recipient);
    }
}

// Injector1: Create a NotificationService with EmailProvider dependency
const emailProvider = new EmailProvider();
const emailService = new NotificationService(emailProvider);
emailService.sendNotification('Hello via Email', 'user@example.com');

// Injector2: Create a NotificationService with SMSProvider dependency
const smsProvider = new SMSProvider();
const smsService = new NotificationService(smsProvider);
smsService.sendNotification('Hello via SMS', '123-456-7890');

```

**Sample Code with DI (Automatic Injection in NestJs)**

**_Interface and Service_**
We'll start by defining the notification providers (EmailProvider and SMSProvider) as services in NestJS. Both the service implements the same interface.

```javascript 
// notification.ts
import { Injectable } from '@nestjs/common';

// NotificationProvider interface
export abstract class NotificationProvider {
  abstract sendNotification(message: string, recipient: string): void;
}

// email.provider.ts
// EmailProvider implementation
@Injectable()
export class EmailProvider implements NotificationProvider {
  sendNotification(message: string, recipient: string): void {
    // Send email logic
    console.log(`Sending email to ${recipient}: ${message}`);
  }
}

// sms.provider.ts
// SMSProvider implementation
@Injectable()
export class SMSProvider implements NotificationProvider {
  sendNotification(message: string, recipient: string): void {
    // Send SMS logic
    console.log(`Sending SMS to ${recipient}: ${message}`);
  }
}

```

**_Client_**
Next, we create the NotificationService that will use NotificationProvider. NestJS will automatically inject the correct implementation (either EmailProvider or SMSProvider), based on the provider configuration in the module.

```javascript
import { Injectable } from '@nestjs/common';
import { NotificationProvider } from './notification.provider'; // Import NotificationProvider interface

@Injectable()
export class NotificationService {
  constructor(
    private readonly notificationProvider: NotificationProvider, // NestJS will inject the appropriate provider here
  ) {}

  sendNotification(message: string, recipient: string): void {
    this.notificationProvider.sendNotification(message, recipient);
  }
}

```

**_Injector_**
Now, in your module, you'll configure the providers so that NestJS knows how to inject the correct instance into the NotificationService.

We can use useClass to bind the NotificationProvider to either EmailProvider or SMSProvider. 

In this example, we are telling NestJS to inject EmailProvider when NotificationProvider is requested. If you want to use the SMSProvider instead, just change useClass: SMSProvider.

```javascript
import { Module } from '@nestjs/common';
import { NotificationService } from './notification.service';
import { EmailProvider } from './email.provider';
import { SMSProvider } from './sms.provider';

@Module({
  providers: [
    NotificationService,
    {
      provide: NotificationProvider,
      useClass: EmailProvider, // You can swap this to SMSProvider for different behavior
    },
  ],
  exports: [NotificationService], // Export the service if you need it in other modules
})
export class NotificationModule {}

```

**_Calling the Service_**
Now that everything is set up, you can use NotificationService in other parts of your application. NestJS will automatically inject the correct provider.

```javascript
import { Controller, Get } from '@nestjs/common';
import { NotificationService } from './notification.service';

@Controller()
export class AppController {
  constructor(private readonly notificationService: NotificationService) {}

  @Get('send-email')
  sendEmail() {
    this.notificationService.sendNotification('Hello via Email', 'user@example.com');
  }

  @Get('send-sms')
  sendSMS() {
    this.notificationService.sendNotification('Hello via SMS', '123-456-7890');
  }
}

```


#### Why do I need it?

**_Resilence_**: To improve resilience and user experience.

**__**: To avoid failing fast on temporary issues that would resolve on their own (e.g., brief downtime or latency spikes).


#### Use Case:

You're calling a 3rd-party payment API (https://api.payment.com/charge) to charge a customer.
Sometimes the API fails with a timeout or 500 error due to high load.
You don't want to immediately give up — you'd like to retry 3 times before returning an error to the user.


#### Implementation Options

### 7. SAGA Pattern
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

#### Why do I need it?

**_Resilence_**: To improve resilience and user experience.

**__**: To avoid failing fast on temporary issues that would resolve on their own (e.g., brief downtime or latency spikes).


#### Use Case:

xxxxxxxxxxxxxxxxxxxxxxxxxxxx


#### Implementation Options